# K7 Yukawa Tensor Analysis - Full b3=77

**GIFT Framework v2.2** - Complete Yukawa analysis with full H3 cohomology.

This notebook analyzes the Yukawa tensor Y_ijk from the K7 G2 manifold with:
- b2(K7) = 21 harmonic 2-forms (H2 modes)
- b3(K7) = 77 harmonic 3-forms (35 local + 42 global)

**Topological decomposition**:
- Local modes (35): constant fiber forms from Lambda^3(R^7)
- Global modes (42): TCS profile functions from gluing

**GIFT invariants**:
- det(g) = 65/32 = 2.03125 (exact)
- kappa_T = 1/61 = 0.016393

---

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, '../..')

from G2_ML.meta_hodge import (
    CandidateLibrary,
    YukawaExtractor,
    locate_historical_assets,
    load_version_model,
    sample_coords,
)

# GIFT constants
B2_TARGET = 21
B3_TARGET = 77
B3_LOCAL = 35
B3_GLOBAL = 42
DET_G_TARGET = 65.0 / 32.0

print(f"GIFT Structural Constants:")
print(f"  b2(K7) = {B2_TARGET}")
print(f"  b3(K7) = {B3_TARGET} = {B3_LOCAL} (local) + {B3_GLOBAL} (global)")
print(f"  det(g) = 65/32 = {DET_G_TARGET:.6f}")

## 1. Load Data and Compute Yukawa

In [ ]:
# Load v1.6 GIFT-calibrated data
registry = locate_historical_assets()
bundle = load_version_model('1_6', registry)
library = CandidateLibrary(versions=['1_6'], registry=registry)

# Sample points
N_SAMPLES = 1024
x = sample_coords(N_SAMPLES)
g = bundle.metric_fn(x)

print(f"Samples: {x.shape[0]}")
print(f"Metric shape: {g.shape}")

# Get full b3=77 candidate forms
forms77 = library.collect_b3_77(x, version='1_6')

print(f"\nCandidate forms:")
print(f"  c2 (H2 basis): {forms77.c2.shape}")
print(f"  c3_local:      {forms77.c3_local.shape}")
print(f"  c3_global:     {forms77.c3_global.shape}")
print(f"  c3_combined:   {forms77.c3_combined.shape}")

In [ ]:
# Compute Yukawa tensor
extractor = YukawaExtractor(x, g)
Y = extractor.compute(forms77.c2, forms77.c3_combined)

print(f"Yukawa tensor shape: {Y.shape}")
print(f"Expected: ({B2_TARGET}, {B2_TARGET}, {B3_TARGET})")

# Verify metric determinant
det_g = torch.linalg.det(g)
print(f"\nMetric det(g): {det_g.mean():.6f} (target: {DET_G_TARGET:.6f})")
print(f"Deviation: {abs(det_g.mean() - DET_G_TARGET) / DET_G_TARGET * 100:.4f}%")

## 2. Basic Statistics

In [ ]:
print("=" * 60)
print("YUKAWA TENSOR STATISTICS")
print("=" * 60)

# Symmetry check
sym_diff = (Y - Y.transpose(0, 1)).abs().max()
print(f"\nSymmetry: max|Y_ijk - Y_jik| = {sym_diff:.2e}")
print(f"Symmetric: {sym_diff < 1e-10}")

# Basic stats
print(f"\nAmplitude:")
print(f"  Max |Y|: {Y.abs().max():.4f}")
print(f"  Mean |Y|: {Y.abs().mean():.6f}")
print(f"  Nonzero (>1e-4): {(Y.abs() > 1e-4).sum().item()}")
print(f"  Nonzero (>1e-6): {(Y.abs() > 1e-6).sum().item()}")

# Local vs global contribution
Y_local = Y[:, :, :B3_LOCAL]
Y_global = Y[:, :, B3_LOCAL:]

print(f"\nLocal vs Global:")
print(f"  Local (35 modes):  max={Y_local.abs().max():.4f}, mean={Y_local.abs().mean():.6f}")
print(f"  Global (42 modes): max={Y_global.abs().max():.4f}, mean={Y_global.abs().mean():.6f}")

In [ ]:
# Distribution plots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Full distribution
vals = Y.flatten().numpy()
axes[0].hist(vals[vals != 0], bins=100, alpha=0.7, density=True)
axes[0].set_xlabel('Y_ijk')
axes[0].set_ylabel('Density')
axes[0].set_title('Full Yukawa Distribution')

# Local vs global
local_vals = Y_local.flatten().numpy()
global_vals = Y_global.flatten().numpy()
axes[1].hist(local_vals[np.abs(local_vals) > 1e-6], bins=50, alpha=0.6, label='Local', density=True)
axes[1].hist(global_vals[np.abs(global_vals) > 1e-6], bins=50, alpha=0.6, label='Global', density=True)
axes[1].set_xlabel('Y_ijk')
axes[1].set_ylabel('Density')
axes[1].set_title('Local vs Global Modes')
axes[1].legend()

# Log scale
log_vals = np.log10(np.abs(vals[np.abs(vals) > 1e-10]))
axes[2].hist(log_vals, bins=50, alpha=0.7, color='green', density=True)
axes[2].set_xlabel('log10 |Y_ijk|')
axes[2].set_ylabel('Density')
axes[2].set_title('Log-scale Distribution')

plt.tight_layout()
plt.show()

## 3. SVD Analysis - Family Structure

In [ ]:
print("=" * 60)
print("SVD ANALYSIS - FAMILY HIERARCHY")
print("=" * 60)

# Flatten Y to (21*21, 77) and compute SVD
Y_flat = Y.reshape(-1, B3_TARGET)
U, S, Vh = torch.linalg.svd(Y_flat, full_matrices=False)

print(f"\nSVD shape: U={U.shape}, S={S.shape}, Vh={Vh.shape}")

# Effective rank
rank_1e4 = (S > 1e-4).sum().item()
rank_1e6 = (S > 1e-6).sum().item()
print(f"\nEffective rank:")
print(f"  tol=1e-4: {rank_1e4}")
print(f"  tol=1e-6: {rank_1e6}")

# Singular values
print(f"\nTop 10 singular values:")
for i, s in enumerate(S[:10].tolist()):
    print(f"  sigma_{i}: {s:.6f}")

# Family hierarchy ratios
print(f"\nFamily hierarchy (assuming top 3 = generations):")
if S[0] > 1e-10:
    print(f"  m_3 ~ sigma_0 = {S[0]:.4f}")
    print(f"  m_2 ~ sigma_1 = {S[1]:.4f}  (ratio: {S[1]/S[0]:.4f})")
    print(f"  m_1 ~ sigma_2 = {S[2]:.4f}  (ratio: {S[2]/S[0]:.4f})")

In [ ]:
# Singular value plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Linear scale
axes[0].bar(range(min(30, len(S))), S[:30].numpy(), alpha=0.7)
axes[0].set_xlabel('Singular value index')
axes[0].set_ylabel('Singular value')
axes[0].set_title('Yukawa Singular Values (top 30)')
axes[0].axhline(1e-4, color='r', linestyle='--', label='tol=1e-4')
axes[0].legend()

# Log scale
axes[1].semilogy(range(len(S)), S.numpy(), 'o-', markersize=3)
axes[1].set_xlabel('Singular value index')
axes[1].set_ylabel('Singular value (log)')
axes[1].set_title('Singular Value Decay')
axes[1].axhline(1e-4, color='r', linestyle='--', label='tol=1e-4')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Mode-by-Mode Analysis

In [ ]:
# Analyze each H3 mode
mode_strength = []
mode_rank = []

for k in range(B3_TARGET):
    M = Y[:, :, k]
    frob = torch.linalg.norm(M).item()
    s = torch.linalg.svdvals(M)
    rank = (s > 1e-6).sum().item()
    mode_strength.append(frob)
    mode_rank.append(rank)

mode_strength = np.array(mode_strength)
mode_rank = np.array(mode_rank)

print("Mode Analysis Summary:")
print(f"  Strength range: [{mode_strength.min():.4f}, {mode_strength.max():.4f}]")
print(f"  Rank range: [{mode_rank.min()}, {mode_rank.max()}]")

# Top modes
top_modes = np.argsort(mode_strength)[::-1][:10]
print(f"\nTop 10 strongest modes:")
for i, k in enumerate(top_modes):
    region = "local" if k < B3_LOCAL else "global"
    print(f"  {i+1}. k={k:2d} ({region:6s}): ||Y^(k)||_F = {mode_strength[k]:.4f}, rank={mode_rank[k]}")

In [ ]:
# Mode strength plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Strength by mode
colors = ['blue'] * B3_LOCAL + ['orange'] * B3_GLOBAL
axes[0].bar(range(B3_TARGET), mode_strength, color=colors, alpha=0.7)
axes[0].axvline(B3_LOCAL - 0.5, color='black', linestyle='--', label='local|global')
axes[0].set_xlabel('k (H3 mode)')
axes[0].set_ylabel('||Y^(k)||_F')
axes[0].set_title('Mode Strength (blue=local, orange=global)')
axes[0].legend()

# Rank by mode
axes[1].bar(range(B3_TARGET), mode_rank, color=colors, alpha=0.7)
axes[1].axvline(B3_LOCAL - 0.5, color='black', linestyle='--')
axes[1].axhline(3, color='red', linestyle=':', label='rank=3')
axes[1].set_xlabel('k (H3 mode)')
axes[1].set_ylabel('Numerical rank')
axes[1].set_title('Mode Rank')
axes[1].legend()

plt.tight_layout()
plt.show()

# Statistics by region
print(f"\nLocal modes (0-34):")
print(f"  Mean strength: {mode_strength[:B3_LOCAL].mean():.4f}")
print(f"  Max strength:  {mode_strength[:B3_LOCAL].max():.4f}")
print(f"\nGlobal modes (35-76):")
print(f"  Mean strength: {mode_strength[B3_LOCAL:].mean():.4f}")
print(f"  Max strength:  {mode_strength[B3_LOCAL:].max():.4f}")

## 5. Contracted Yukawa Matrix

In [ ]:
# Contract over H3 index to get generation matrix
Y_contracted = Y.sum(dim=-1)  # (21, 21)

print("Contracted Yukawa Matrix Y_ij = sum_k Y_ijk")
print(f"Shape: {Y_contracted.shape}")

# Eigenvalue decomposition
eigvals, eigvecs = torch.linalg.eigh(Y_contracted)

print(f"\nEigenvalues:")
print(f"  Min: {eigvals.min():.4f}")
print(f"  Max: {eigvals.max():.4f}")

# SVD
U_c, S_c, Vh_c = torch.linalg.svd(Y_contracted)
print(f"\nSingular values (top 5): {S_c[:5].tolist()}")
print(f"Effective rank: {(S_c > 1e-6).sum().item()}")

In [ ]:
# Visualize contracted matrix
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Matrix heatmap
im = axes[0].imshow(Y_contracted.numpy(), cmap='RdBu_r', aspect='equal')
axes[0].set_xlabel('j')
axes[0].set_ylabel('i')
axes[0].set_title('Contracted Yukawa Y_ij')
plt.colorbar(im, ax=axes[0])

# Eigenvalue spectrum
axes[1].bar(range(21), eigvals.numpy(), alpha=0.7)
axes[1].set_xlabel('Eigenvalue index')
axes[1].set_ylabel('Eigenvalue')
axes[1].set_title('Eigenvalue Spectrum')
axes[1].axhline(0, color='k', linestyle='--', alpha=0.3)

# Singular values
axes[2].semilogy(range(21), S_c.numpy(), 'o-')
axes[2].set_xlabel('Index')
axes[2].set_ylabel('Singular value')
axes[2].set_title('Singular Values of Contracted Matrix')
axes[2].axhline(1e-6, color='r', linestyle='--')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Block Structure (21 = 3 x 7)

In [ ]:
# The 21 H2 modes potentially correspond to 3 families x 7 types
# Analyze block structure

def analyze_block_structure(M, title="Matrix"):
    """Analyze 21x21 matrix as 3x3 blocks of 7x7."""
    block_size = 7
    block_norms = np.zeros((3, 3))
    
    for bi in range(3):
        for bj in range(3):
            block = M[bi*block_size:(bi+1)*block_size, bj*block_size:(bj+1)*block_size]
            block_norms[bi, bj] = np.linalg.norm(block)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Full matrix with grid
    im = axes[0].imshow(M, cmap='RdBu_r', aspect='equal')
    for i in range(1, 3):
        axes[0].axhline(i*block_size - 0.5, color='k', linewidth=1)
        axes[0].axvline(i*block_size - 0.5, color='k', linewidth=1)
    axes[0].set_title(f'{title} (21x21)')
    plt.colorbar(im, ax=axes[0])
    
    # Block norms
    im2 = axes[1].imshow(block_norms, cmap='viridis', aspect='equal')
    axes[1].set_xticks(range(3))
    axes[1].set_yticks(range(3))
    axes[1].set_xticklabels(['Gen 0', 'Gen 1', 'Gen 2'])
    axes[1].set_yticklabels(['Gen 0', 'Gen 1', 'Gen 2'])
    axes[1].set_title('Block Norms (3x3)')
    for i in range(3):
        for j in range(3):
            axes[1].text(j, i, f'{block_norms[i,j]:.2f}', ha='center', va='center', color='white')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
    
    return block_norms

# Analyze contracted matrix
print("Block structure of contracted Yukawa:")
block_norms = analyze_block_structure(Y_contracted.numpy(), "Contracted Y_ij")

In [ ]:
# Analyze strongest H3 mode
k_strongest = top_modes[0]
print(f"\nBlock structure of strongest mode k={k_strongest}:")
M_strong = Y[:, :, k_strongest].numpy()
block_norms_strong = analyze_block_structure(M_strong, f"Y^({k_strongest})")

## 7. Summary

In [ ]:
print("=" * 70)
print("YUKAWA TENSOR ANALYSIS SUMMARY - FULL b3=77")
print("=" * 70)

print(f"\n1. TENSOR STRUCTURE")
print(f"   Shape: {Y.shape}")
print(f"   H2 modes: {B2_TARGET} (harmonic 2-forms)")
print(f"   H3 modes: {B3_TARGET} = {B3_LOCAL} (local) + {B3_GLOBAL} (global)")
print(f"   Total entries: {Y.numel()}")
print(f"   Nonzero (>1e-4): {(Y.abs() > 1e-4).sum().item()}")
print(f"   Symmetry Y_ijk = Y_jik: {sym_diff < 1e-10}")

print(f"\n2. AMPLITUDE")
print(f"   Max |Y|: {Y.abs().max():.4f}")
print(f"   Mean |Y|: {Y.abs().mean():.6f}")
print(f"   Local max: {Y_local.abs().max():.4f}")
print(f"   Global max: {Y_global.abs().max():.4f}")

print(f"\n3. SVD / FAMILY STRUCTURE")
print(f"   Effective rank: {rank_1e4} (tol=1e-4)")
print(f"   Top 3 singular values: {S[:3].tolist()}")
if S[0] > 1e-10:
    print(f"   Ratios: sigma_1/sigma_0 = {S[1]/S[0]:.4f}, sigma_2/sigma_0 = {S[2]/S[0]:.4f}")

print(f"\n4. DOMINANT MODES")
for i, k in enumerate(top_modes[:5]):
    region = "local" if k < B3_LOCAL else "global"
    print(f"   {i+1}. k={k:2d} ({region}): ||Y^(k)||_F = {mode_strength[k]:.4f}")

print(f"\n5. GIFT INVARIANTS (from metric)")
print(f"   det(g) = {det_g.mean():.6f} (target: {DET_G_TARGET:.6f})")
print(f"   b2(K7) = {B2_TARGET}, b3(K7) = {B3_TARGET}")

print("\n" + "=" * 70)